In [2]:
import pandas as pd
import numpy as np

# Load the Excel file
file_path = "occupation.xlsx"  # Replace with your actual filename if different
df = pd.read_excel(file_path)

# Step 1: Replace '—' and other non-numeric entries with NaN
wage_col = 'Median annual wage, dollars, 2024[1]'
df[wage_col] = df[wage_col].replace('—', np.nan)

# Step 2: Remove dollar signs and commas
df[wage_col] = df[wage_col].replace('[\$,]', '', regex=True)

# Step 3: Convert to float
df[wage_col] = pd.to_numeric(df[wage_col], errors='coerce')

# Step 4: Filter in-demand jobs
filtered = df[
    (df['Employment change, percent, 2023-33'] > 3) &
    (df['Occupational openings, 2023-33 annual average'] >= 900) &
    (df[wage_col] > 30000)
]

# Step 5: Select relevant columns
result = filtered[[
    '2023 National Employment Matrix title',
    '2023 National Employment Matrix code',
    'Employment change, percent, 2023-33',
    'Occupational openings, 2023-33 annual average',
    wage_col
]]

# Show top 10 results
print(result.head(11))

result.to_csv('filtered_jobs.csv', index=False)



                2023 National Employment Matrix title  \
0                              Total, all occupations   
1     Food preparation and serving related occupat...   
3      Transportation and material moving occupations   
5                   Food and beverage serving workers   
6                      Healthcare support occupations   
8                              Management occupations   
9                             Material moving workers   
10                       Laborers and material movers   
12      Business and financial operations occupations   
13      Home health and personal care aides; and n...   
14                      Fast food and counter workers   

   2023 National Employment Matrix code  Employment change, percent, 2023-33  \
0                               00-0000                                  4.0   
1                               35-0000                                  4.3   
3                               53-0000                                  4.

In [3]:
import pandas as pd

# Load filtered jobs CSV
jobs_df = pd.read_csv('filtered_jobs.csv')

print(jobs_df.columns)  # Check columns

# Extract SOC codes as strings and strip
target_soc_codes = jobs_df['2023 National Employment Matrix code'].astype(str).tolist()
target_soc_codes = [code.strip() for code in target_soc_codes]

# Add decimal suffix if missing to match O*NET format
def format_soc_code(code):
    if '.' not in code:
        return code + '.00'
    return code

target_soc_codes = [format_soc_code(code) for code in target_soc_codes]

print(target_soc_codes)  # Confirm updated SOC codes

# Load O*NET skills data
skills_df = pd.read_excel('Skills.xlsx')

# Filter for Importance ratings only
importance_df = skills_df[skills_df['Scale ID'] == 'IM']

# Filter for your occupations with formatted SOC codes
importance_df = importance_df[importance_df['O*NET-SOC Code'].isin(target_soc_codes)]

# Filter for skills with Importance > 2
important_skills = importance_df[importance_df['Data Value'] > 2]

# Group skills by occupation code and list skill names
skills_by_job = important_skills.groupby('O*NET-SOC Code')['Element Name'].apply(list)

print(skills_by_job)


Index(['2023 National Employment Matrix title',
       '2023 National Employment Matrix code',
       'Employment change, percent, 2023-33',
       'Occupational openings, 2023-33 annual average',
       'Median annual wage, dollars, 2024[1]'],
      dtype='object')
['00-0000.00', '35-0000.00', '53-0000.00', '35-3000.00', '31-0000.00', '11-0000.00', '53-7000.00', '53-7060.00', '13-0000.00', '31-1100.00', '35-3023.00']
O*NET-SOC Code
35-3023.00    [Reading Comprehension, Active Listening, Writ...
Name: Element Name, dtype: object


In [4]:
import pandas as pd

# Load filtered jobs CSV
jobs_df = pd.read_csv('filtered_jobs.csv')

print(jobs_df.columns)  # Check columns

# Extract SOC codes as strings and strip whitespace
target_soc_codes = jobs_df['2023 National Employment Matrix code'].astype(str).tolist()
target_soc_codes = [code.strip() for code in target_soc_codes]

# Add decimal suffix if missing to match O*NET format
def format_soc_code(code):
    if '.' not in code:
        return code + '.00'
    return code

target_soc_codes = [format_soc_code(code) for code in target_soc_codes]

print("Original SOC codes:", target_soc_codes)

# Load O*NET skills data
skills_df = pd.read_excel('Skills.xlsx')

# Get all unique SOC codes in the O*NET dataset
all_onet_codes = skills_df['O*NET-SOC Code'].unique().tolist()

# Function to expand broad SOC codes to detailed codes
def get_detailed_codes(general_code, all_codes):
    # Remove trailing '-0000.00' to get prefix (first two digits + '-')
    prefix = general_code.split('-')[0] + '-'
    # Select detailed codes that start with prefix and do NOT end with '0000.00'
    detailed_codes = [code for code in all_codes if code.startswith(prefix) and not code.endswith('0000.00')]
    return detailed_codes

# Expand broad SOC codes to detailed SOC codes
expanded_soc_codes = []
for soc in target_soc_codes:
    if soc.endswith('0000.00'):
        detailed_list = get_detailed_codes(soc, all_onet_codes)
        if detailed_list:
            expanded_soc_codes.extend(detailed_list)
        else:
            # fallback: keep the broad code if no detailed found
            expanded_soc_codes.append(soc)
    else:
        expanded_soc_codes.append(soc)

# Remove duplicates
expanded_soc_codes = list(set(expanded_soc_codes))

print("Expanded SOC codes count:", len(expanded_soc_codes))

# Filter O*NET data for Importance ratings only
importance_df = skills_df[skills_df['Scale ID'] == 'IM']

# Filter for expanded SOC codes
importance_df = importance_df[importance_df['O*NET-SOC Code'].isin(expanded_soc_codes)]

# Filter for Importance > 2
important_skills = importance_df[importance_df['Data Value'] > 2]

# Group skills by detailed SOC code
skills_by_job = important_skills.groupby('O*NET-SOC Code')['Element Name'].apply(list)

print(skills_by_job)


Index(['2023 National Employment Matrix title',
       '2023 National Employment Matrix code',
       'Employment change, percent, 2023-33',
       'Occupational openings, 2023-33 annual average',
       'Median annual wage, dollars, 2024[1]'],
      dtype='object')
Original SOC codes: ['00-0000.00', '35-0000.00', '53-0000.00', '35-3000.00', '31-0000.00', '11-0000.00', '53-7000.00', '53-7060.00', '13-0000.00', '31-1100.00', '35-3023.00']
Expanded SOC codes count: 181
O*NET-SOC Code
11-1011.00    [Reading Comprehension, Active Listening, Writ...
11-1011.03    [Reading Comprehension, Active Listening, Writ...
11-1021.00    [Reading Comprehension, Active Listening, Writ...
11-2011.00    [Reading Comprehension, Active Listening, Writ...
11-2021.00    [Reading Comprehension, Active Listening, Writ...
                                    ...                        
53-7071.00    [Reading Comprehension, Active Listening, Writ...
53-7072.00    [Reading Comprehension, Active Listening, Writ...
5

In [5]:
for soc_code, skills_list in skills_by_job.items():
    print(f"{soc_code}:")
    for skill in skills_list:
        print(f" - {skill}")
    print()  # blank line between SOC codes

11-1011.00:
 - Reading Comprehension
 - Active Listening
 - Writing
 - Speaking
 - Mathematics
 - Critical Thinking
 - Active Learning
 - Learning Strategies
 - Monitoring
 - Social Perceptiveness
 - Coordination
 - Persuasion
 - Negotiation
 - Instructing
 - Service Orientation
 - Complex Problem Solving
 - Operations Analysis
 - Judgment and Decision Making
 - Systems Analysis
 - Systems Evaluation
 - Time Management
 - Management of Financial Resources
 - Management of Material Resources
 - Management of Personnel Resources

11-1011.03:
 - Reading Comprehension
 - Active Listening
 - Writing
 - Speaking
 - Mathematics
 - Science
 - Critical Thinking
 - Active Learning
 - Learning Strategies
 - Monitoring
 - Social Perceptiveness
 - Coordination
 - Persuasion
 - Negotiation
 - Instructing
 - Service Orientation
 - Complex Problem Solving
 - Operations Analysis
 - Judgment and Decision Making
 - Systems Analysis
 - Systems Evaluation
 - Time Management
 - Management of Financial Resou